# Vignette of plotting the likelihood of pop size estimate
In this vignette you will learn how to estimate effective pop sizes from ROH in specific length bins. This model assumes as panmictic population, so think of this estimates as the size of the ancestry pool at the time depth of the co-ancestry of the ROH you are fitting. The model used is described in the Supplement of https://doi.org/10.1101/2020.06.01.126730 and using the same framework introduced in https://www.genetics.org/content/205/3/1335

@author: Harald Ringbauer

In [11]:
import os as os
import pandas as pd
import numpy as np
import sys

## Set the path
You can set the path here to the path you want to work in (relative data loads will be done from there)

In [2]:
### Fill in your own path here!
path = "/mnt/archgen/users/hringbauer/git/hapROH/Notebooks/Vignettes/"  # The Path to Package Midway Cluster
os.chdir(path)  # Set the right Path (in line with Atom default)
print(f"Set path to: {os.getcwd()}") # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns

Set path to: /mnt/archgen/users/hringbauer/git/hapROH/Notebooks/Vignettes


# Load the data for the MLE analysis
We can analyse any set of individuals (including sets of a single individual). One has to load the ROH of these (as a list of lists), and passes them on to the MLE object.

The example ROH data here is hapROH output from the `./callROH_vignette` for 8 Levant Neolithic Chalcolithic individuals, run on Eigenstrat pseudo-haploid aDNA data that can be downloaded from `https://reich.hms.harvard.edu/sites/reich.hms.harvard.edu/files/inline-files/Levant_ChL.tar.gz`. The code here can be easily adapted to the ROH output from any set of indivdiuals.

In [3]:
sys.path.insert(0, "/mnt/archgen/users/hringbauer/git/hapROH/package")  ### Leipzig  Development Version Harald
from hapsburg.PackagesSupport.fit_ne import MLE_ROH_Ne, load_roh_vec

In [4]:
df1 = pd.read_csv("./ExampleOutput/combined_roh05.csv", sep='\t')
df2 = df1[df1["sum_roh>20"]<50] # Remove close-kin consanguineous individuals - 50 cm in total of very long ROH (>20cm) is the cutoff chosen here.
iids = df2["iid"].values # Load list of all iids
print(f"Loaded {len(iids)} IIDs")

Loaded 8 IIDs


In [5]:
iids

array(['I1170', 'I1166', 'I1160', 'I0644', 'I1152', 'I1168', 'I1165',
       'I1182'], dtype=object)

### Load ROH to analyze
This step requires that the individuals to analyse have been run with hapROH. Check the `callROH_vignette`
It loads all ROH lengths - in the step below you can define which lengths to actually fit.

In [6]:
roh_vec = load_roh_vec(iids=iids, base_path = "./ExampleOutput/", suffix="_roh_full.csv")

# Use Maximum Likelihood to fit Ne
Having the roh_vec loaded, now we can fit Ne. We use a class implemented in hapROH for that purpose. This retuns a pandas dataframe, with `coef` giving the most likely estimate, and two columns for the lower and upper bound of the 95% CI intervall for the estimate.

Note: Depending on demography (how many generations are alive, skew of reproductive success), the ratio of effective to census size is often **0.1-0.3x**. So the census size can be 3-10x bigger than the effective size! Check the popgen literature for further details.

Important: By default the reported estimates are for 2Ne. You have to divide by 2 to get the estimates in Ne.

In [7]:
%%time
output = True
min_len = 4 # Min ROH length in cM to fit
max_len = 20 # Max ROH length in cM to fit

mle = MLE_ROH_Ne(start_params=1000, endog=roh_vec,
                 min_len=min_len, max_len=max_len,
                 chr_lgts=[],      # lengths of Chromosomes to fit (in cM). If len 0, use default for 1240K
                 error_model=False, output=False)
fit = mle.fit_ll_profile()

mle.summary/2  # to get estimates in terms of Ne

CPU times: user 678 ms, sys: 1.95 ms, total: 680 ms
Wall time: 678 ms


,coef,std err,0.025,0.975,n
0,5284.379856,NaN,2855.793239,11543.389971,4.0


Ne=5284 (2855-11543 95% CI), that's a value typical for relatively large populations. It is a value where little ROH, even in the 4-8 cM category, as expected.

### [Advanced Parameter]
The default values for the parameter search in fit_ll_profile() is ns = np.logspace(2,5,1000)
ns is an array that specficies which 2Ne values to try (and look for the maximumg and 95%CI)

One can update the ns array to other values. That is for instance useful if the upper CI or even estimate hits 2N=100,000, the default limit.

# Alternative Mode for CI Intervalls
In the above method 95% Confidence Intervalls are fitted via the likelihood profile (calculating the log likelihood for a large number of 2Ne, and looking for the interval 1.92 LL units down from the Maximum Likelihood.)

A simpler and quicker way is to use the curvature of the likelihood (the so called Fisher Information matrix). This works well for a lot of data and small pop sizes - as the likelihood is approxiamted well by this Gaussian fit. However, when the likelihood is "flat", it is better to use the above method.

In [8]:
%%time
output = True
min_len = 4 # Min ROH length in cM to fit
max_len = 20 # Max ROH length in cM to fit

mle = MLE_ROH_Ne(start_params=1000, endog=roh_vec,
                 min_len=4, max_len=20,
                 chr_lgts=[],      # lengths of Chromosomes to fit (in cM). If len 0, use default for 1240K
                 error_model=False, output=False)
fit = mle.fit()
#summary = fit.summary()
mle.summary/2  # to get estimates in terms of Ne

Optimization terminated successfully.
         Current function value: 4.788653
         Iterations: 33
         Function evaluations: 67
CPU times: user 63.9 ms, sys: 3.09 ms, total: 67 ms
Wall time: 112 ms


/mnt/archgen/users/hringbauer/pyenvHPC/lib/python3.12/site-packages/statsmodels/base/model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
/mnt/archgen/users/hringbauer/pyenvHPC/lib/python3.12/site-packages/statsmodels/base/model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")
/mnt/archgen/users/hringbauer/git/hapROH/package/hapsburg/PackagesSupport/fit_ne.py:305: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results_as_html, header=0, index_col=0)[0]


,coef,std err,0.025,0.975
const,5295.0,1877.148,1617.463,9000.0


The estimates agree, but note that the CI are different. The "symmetric" and local approximation capture the order of magnitude of uncertainty, but is problematic in this case.

# Further Application: Fitting IBD_X
You can also use this machinery to fit pop sizes from IBD on the X. You have to set the chromosome length to chr_lgts=[180.85 * 2/3,], and first convert the length of the IBD_X to the sex-averaged rate (2/3 * (length of IBD in female map units)

In [12]:
ns = np.logspace(2,5,num=1000)
ns

array([   100.        ,    100.69386315,    101.39254076,    102.09606623,
          102.80447321,    103.51779556,    104.2360674 ,    104.95932306,
          105.68759712,    106.42092441,    107.15933998,    107.90287915,
          108.65157747,    109.40547072,    110.16459496,    110.92898649,
          111.69868185,    112.47371784,    113.25413152,    114.0399602 ,
          114.83124145,    115.62801312,    116.43031329,    117.23818033,
          118.05165286,    118.87076977,    119.69557024,    120.52609369,
          121.36237983,    122.20446866,    123.05240044,    123.90621569,
          124.76595526,    125.63166025,    126.50337204,    127.38113232,
          128.26498305,    129.1549665 ,    130.05112522,    130.95350205,
          131.86214014,    132.77708294,    133.69837418,    134.62605793,
          135.56017853,    136.50078065,    137.44790927,    138.40160966,
          139.36192742,    140.32890848,    141.30259906,    142.28304572,
          143.27029534,  